In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


In [3]:
train_dir = "../dataset/train"
val_dir = "../dataset/val"

datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

val_data = datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

print(train_data.class_indices)


Found 1096 images belonging to 5 classes.
Found 1096 images belonging to 5 classes.
{'normal': 0, 'vitamin_A': 1, 'vitamin_B': 2, 'vitamin_C': 3, 'vitamin_D': 4}


In [4]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
output = Dense(5, activation="softmax")(x)  # 5 classes

model = Model(inputs=base_model.input, outputs=output)


In [5]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [6]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


Epoch 1/10
35/35 [==============================] - 139s 4s/step - loss: 1.3607 - accuracy: 0.4790 - val_loss: 1.2294 - val_accuracy: 0.6670
Epoch 2/10
35/35 [==============================] - 139s 4s/step - loss: 1.1996 - accuracy: 0.6058 - val_loss: 1.1448 - val_accuracy: 0.7053
Epoch 3/10
35/35 [==============================] - 138s 4s/step - loss: 1.1105 - accuracy: 0.6332 - val_loss: 1.0561 - val_accuracy: 0.6852
Epoch 4/10
35/35 [==============================] - 139s 4s/step - loss: 1.0856 - accuracy: 0.6405 - val_loss: 1.0244 - val_accuracy: 0.7080
Epoch 5/10
35/35 [==============================] - 89s 3s/step - loss: 1.0204 - accuracy: 0.6670 - val_loss: 1.0052 - val_accuracy: 0.7053
Epoch 6/10
35/35 [==============================] - 89s 3s/step - loss: 0.9593 - accuracy: 0.6825 - val_loss: 0.9603 - val_accuracy: 0.6779
Epoch 7/10
35/35 [==============================] - 89s 3s/step - loss: 0.9441 - accuracy: 0.6971 - val_loss: 0.9348 - val_accuracy: 0.7089
Epoch 8/10
35/35

In [12]:
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False
    else:
        layer.trainable = True


In [13]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=1e-6),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [14]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)



Epoch 1/5
35/35 [==============================] - 187s 5s/step - loss: 0.6720 - accuracy: 0.7737 - val_loss: 0.4697 - val_accuracy: 0.8531
Epoch 2/5
35/35 [==============================] - 181s 5s/step - loss: 0.4221 - accuracy: 0.8522 - val_loss: 0.3348 - val_accuracy: 0.8841
Epoch 3/5
35/35 [==============================] - 182s 5s/step - loss: 0.3688 - accuracy: 0.8668 - val_loss: 0.3957 - val_accuracy: 0.8759
Epoch 4/5
35/35 [==============================] - 182s 5s/step - loss: 0.5858 - accuracy: 0.7974 - val_loss: 0.4607 - val_accuracy: 0.8385
Epoch 5/5
35/35 [==============================] - 183s 5s/step - loss: 0.4211 - accuracy: 0.8604 - val_loss: 0.4406 - val_accuracy: 0.8613


In [15]:
model.save("../models/resnet50_finetuned.h5")
print("ResNet50 fine-tuned model saved!")


ResNet50 fine-tuned model saved!
